# Important libraries

In [1]:
import win32com.client
import sys
import time
import os
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Openserver Setup

Openserver class with methods to connect to the server, send commands, and disconnect from the server.

In [2]:
class OpenServer():
    "Class for holding ActiveX reference. Allows license disconnection"
    def __init__(self):
        self.status = "Disconnected"
        self.OSReference = None
    
    def Connect(self):
        self.OSReference = win32com.client.Dispatch("PX32.OpenServer.1")
        self.status = "Connected"
        print("OpenServer connected")
        
    def Disconnect(self):
        self.OSReference = None
        self.status = "Disconnected"
        print("OpenServer disconnected")

In [3]:
def GetAppName(sv):
    # function for returning app name from tag string
    pos = sv.find(".")
    if pos < 2:
        sys.exit("GetAppName: Badly formed tag string")
    app_name = sv[:pos]
    if app_name.lower() not in ["prosper", "mbal", "gap", "pvt", "resolve",
                                   "reveal"]:
        sys.exit("GetAppName: Unrecognised application name in tag string")
    return app_name


def DoCmd(OpenServe, cmd):
    # perform a command and check for errors
    lerr = OpenServe.OSReference.DoCommand(cmd)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoCmd: " + err)


def DoSet(OpenServe, sv, val):
    # set a value and check for errors
    lerr = OpenServe.OSReference.SetValue(sv, val)
    app_name = GetAppName(sv)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSet: " + err)
    
def DoGet(OpenServe, gv):
    # get a value and check for errors
    get_value = OpenServe.OSReference.GetValue(gv)
    app_name = GetAppName(gv)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoGet: " + err)
    return get_value


def DoSlowCmd(OpenServe, cmd):
    # perform a command then wait for command to exit and check for errors
    step = 0.001
    app_name = GetAppName(cmd)
    lerr = OpenServe.OSReference.DoCommandAsync(cmd)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSlowCmd: " + err)
    while OpenServe.OSReference.IsBusy(app_name) > 0:
        if step < 2:
            step = step*2
        time.sleep(step)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSlowCmd: " + err)


def DoGAPFunc(OpenServe, gv):
    DoSlowCmd(gv)
    DoGAPFunc = DoGet(OpenServe, "GAP.LASTCMDRET")
    lerr = OpenServe.OSReference.GetLastError("GAP")
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoGAPFunc: " + err)
    return DoGAPFunc


def OSOpenFile(OpenServe, theModel, appname):
    DoSlowCmd(OpenServe, appname + '.OPENFILE ("' + theModel + '")')
    lerr = OpenServe.OSReference.GetLastError(appname)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("OSOpenFile: " + err)


def OSSaveFile(OpenServe, theModel, appname):
    DoSlowCmd(OpenServe, appname + '.SAVEFILE ("' + theModel + '")')
    lerr = OpenServe.OSReference.GetLastError(appname)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("OSSaveFile: " + err)

# Multiple well calculation

In [4]:
#gas availability and increment steps
number_of_wells = int(input('Please enter the number of wells:'))
gas_available = float(input('Please enter available gas'))
increment = float(input('Please enter the increment value'))
gas_injection_array = np.arange(0, gas_available, increment)
df=pd.DataFrame()
df['gas_injection_array'] = gas_injection_array
df

,gas_injection_array
0,0.0
1,0.1
2,0.2
3,0.3
4,0.4
5,0.5
6,0.6
7,0.7
8,0.8
9,0.9


In [5]:
# Initialises an 'OpenServer' class

petex = OpenServer()

# Creates ActiveX reference and holds a license

petex.Connect()
oil_rates_for_wells = {}

# Perform functions

cwd = os.getcwd()
for j in range(1,number_of_wells+1):
    #opening well file
    OSOpenFile(petex, cwd + f'\models\well_{j}.Out', 'PROSPER')
    print(f'Well {j} opened')
    #oil rates calculation
    for i in range(0,len(gas_injection_array)):
        command = f'PROSPER.ANL.SYS.Sens.SensDB.Sens[138].Vals[{i}]'
        DoSet(petex, command,gas_injection_array[i])
    DoCmd(petex, 'PROSPER.ANL.SYS.CALC')
    oil_rates = [] #list of oil rates for 1 well
    for i in range(0, len(gas_injection_array)):
        value = f'PROSPER.OUT.SYS.Results[{i}].Sol.OilRate'
        oil_rates.append(np.round(float(DoGet(petex, value)),2))
    oil_rates_for_wells[f'well_{j}'] = oil_rates
    #closing file   
    OSSaveFile(petex, cwd + f'\well_{j}.Out', 'PROSPER')
    print(f'Well {j} closed')

#merging to main dataframe
for i in range(1,number_of_wells+1):
    df[f'Well_{i}'] = oil_rates_for_wells[f'well_{i}']

df.head()

OpenServer connected
Well 1 opened
Well 1 closed
Well 2 opened
Well 2 closed
Well 3 opened
Well 3 closed
Well 4 opened
Well 4 closed
Well 5 opened
Well 5 closed
Well 6 opened
Well 6 closed
Well 7 opened
Well 7 closed
Well 8 opened
Well 8 closed
Well 9 opened
Well 9 closed
Well 10 opened
Well 10 closed


,gas_injection_array,Well_1,Well_2,Well_3,Well_4,Well_5,Well_6,Well_7,Well_8,Well_9,Well_10
0,0.0,878.37,1657.35,1576.44,1362.42,997.05,1858.40,1767.62,1123.93,1717.52,1749.18
1,0.1,896.05,1668.40,1588.09,1375.90,1013.68,1867.23,1777.72,1139.83,1728.00,1759.44
2,0.2,911.90,1677.71,1597.90,1387.41,1027.60,1874.69,1786.06,1153.42,1739.01,1767.97
3,0.3,924.19,1685.58,1606.21,1397.28,1041.02,1880.97,1793.10,1164.63,1746.45,1775.18
4,0.4,935.19,1692.24,1613.28,1405.79,1051.22,1886.24,1799.04,1174.25,1752.74,1781.27


In [6]:
fig = go.Figure()
for i in range(1,number_of_wells+1):
    
    fig.add_trace(go.Scatter(x=df['gas_injection_array'], y=df[f'Well_{i}'],\
                            mode='lines', name=f'Well_{i}'))
fig.update_layout(title='Sensitivity Plot',
                   xaxis_title='Gaslift Gas Injection Rate(MMscf/day)',
                   yaxis_title='Oil Rate(STB/day)',
                   title_x=0.5, 
                   )
fig.update_xaxes(nticks=50)
fig.update_yaxes(nticks=20)
fig.show()

In [7]:
df.head()

,gas_injection_array,Well_1,Well_2,Well_3,Well_4,Well_5,Well_6,Well_7,Well_8,Well_9,Well_10
0,0.0,878.37,1657.35,1576.44,1362.42,997.05,1858.40,1767.62,1123.93,1717.52,1749.18
1,0.1,896.05,1668.40,1588.09,1375.90,1013.68,1867.23,1777.72,1139.83,1728.00,1759.44
2,0.2,911.90,1677.71,1597.90,1387.41,1027.60,1874.69,1786.06,1153.42,1739.01,1767.97
3,0.3,924.19,1685.58,1606.21,1397.28,1041.02,1880.97,1793.10,1164.63,1746.45,1775.18
4,0.4,935.19,1692.24,1613.28,1405.79,1051.22,1886.24,1799.04,1174.25,1752.74,1781.27


In [8]:
#extracting max gas lift oil rate pairs
df_new = df.drop('gas_injection_array', axis=1)
wells = []
i=1
for element in df_new.idxmax():
    oil_rate = df_new[f'Well_{i}'][element]
    gas_lift = df['gas_injection_array'][element]
    wells.append({'production_rate':oil_rate, 'gas_lift':gas_lift})
    i+=1
wells

[{'production_rate': 987.44, 'gas_lift': 1.7000000000000002},
 {'production_rate': 1717.55, 'gas_lift': 1.4000000000000001},
 {'production_rate': 1643.5, 'gas_lift': 1.5},
 {'production_rate': 1448.2, 'gas_lift': 1.7000000000000002},
 {'production_rate': 1104.21, 'gas_lift': 1.9000000000000001},
 {'production_rate': 1904.17, 'gas_lift': 1.2000000000000002},
 {'production_rate': 1819.56, 'gas_lift': 1.3},
 {'production_rate': 1225.93, 'gas_lift': 1.9000000000000001},
 {'production_rate': 1775.91, 'gas_lift': 1.3},
 {'production_rate': 1802.58, 'gas_lift': 1.3}]

In [1]:
wells=[{'production_rate': 987.44, 'gas_lift': 1.7000000000000002},
 {'production_rate': 1717.55, 'gas_lift': 1.4000000000000001},
 {'production_rate': 1643.5, 'gas_lift': 1.5},
 {'production_rate': 1448.2, 'gas_lift': 1.7000000000000002},
 {'production_rate': 1104.21, 'gas_lift': 1.9000000000000001},
 {'production_rate': 1904.17, 'gas_lift': 1.2000000000000002},
 {'production_rate': 1819.56, 'gas_lift': 1.3},
 {'production_rate': 1225.93, 'gas_lift': 1.9000000000000001},
 {'production_rate': 1775.91, 'gas_lift': 1.3},
 {'production_rate': 1802.58, 'gas_lift': 1.3}]

# Ant Colony Algorithm

In [28]:
import numpy as np

def optimize_gas_allocation(wells, gas_capacity, alpha, beta, evaporation_rate, iteration_count):
    """
    Optimizes the allocation of gas to a group of wells using an ant colony algorithm.
    
    Parameters:
    - wells: a list of dictionaries representing the wells, where each dictionary contains the following keys:
        - 'production_rate': the production rate of the well (in STB/day)
        - 'gas_lift': the gas lift of the well (in STB/day)
    - gas_capacity: the total gas capacity available for allocation (in STB/day)
    - alpha: the weight of the pheromone trail in the decision making process
    - beta: the weight of the heuristic value in the decision making process
    - evaporation_rate: the rate at which the pheromone trails evaporate
    - iteration_count: the number of iterations to run the algorithm for
    
    Returns:
    - A list of dictionaries representing the optimized allocation of gas to the wells, where each dictionary contains the following keys:
        - 'well_id': the ID of the well
        - 'allocation': the optimized allocation of gas to the well (in STB/day)
    """
    # Initialize the pheromone trails and heuristic values for each well
    pheromone_trails = np.ones(len(wells))
    heuristic_values = np.array([well['gas_lift'] / well['production_rate'] for well in wells])
    
    # Run the algorithm for the specified number of iterations
    for i in range(iteration_count):
        # Initialize the allocation for each well
        allocations = np.zeros(len(wells))
        
        # Allocate gas to each well using the ant colony algorithm
        remaining_gas = gas_capacity
        for j, well in enumerate(wells):
            # Calculate the allocation probability for the well
            allocation_probability = (pheromone_trails[j] ** alpha) * (heuristic_values[j] ** beta)
            allocation_probability /= np.sum((pheromone_trails ** alpha) * (heuristic_values ** beta))
            
            # Allocate gas to the well based on the allocation probability
            allocation = remaining_gas * allocation_probability
            allocations[j] = allocation
            remaining_gas -= allocation
            
        # Evaporate the pheromone trails based on the evaporation rate
        pheromone_trails *= (1 - evaporation_rate)
        
        # Update the pheromone trails based on the allocations
        for j, well in enumerate(wells):
            pheromone_trails[j] += allocations[j] / well['production_rate']
    
    # Return the optimized allocation of gas to the wells
    return [{'well_id': j, 'allocation': allocations[j]} for j in range(len(wells))]
   


# Testing

In [29]:
# Define the test case
gas_capacity = gas_available
alpha = 1
beta = 2
evaporation_rate = 0.1
iteration_count = 70

# Optimize the allocation of gas to the wells
allocations = optimize_gas_allocation(wells, gas_capacity, alpha, beta, evaporation_rate, iteration_count)

# Print the optimized allocation
print(allocations)


[{'well_id': 0, 'allocation': 2.7958451134938347}, {'well_id': 1, 'allocation': 0.0050436896972078345}, {'well_id': 2, 'allocation': 0.006686640911786596}, {'well_id': 3, 'allocation': 0.013888613395527683}, {'well_id': 4, 'allocation': 0.06857443284524403}, {'well_id': 5, 'allocation': 0.002256289796639514}, {'well_id': 6, 'allocation': 0.0029716069376196064}, {'well_id': 7, 'allocation': 0.02706769067572858}, {'well_id': 8, 'allocation': 0.002938504307540259}, {'well_id': 9, 'allocation': 0.002818506750440314}]


In [27]:
sum = 0
for i in range(len(allocations)):
    sum+=allocations[i]['allocation']
print(sum)

2.9280910888115694


In [30]:
import numpy as np

def optimize_gas_allocation(wells, gas_capacity, alpha, beta, evaporation_rate, iteration_count):
    """
    Optimizes the allocation of gas to a group of wells in order to maximize the total production of the wells, using an ant colony algorithm.
    
    Parameters:
    - wells: a list of dictionaries representing the wells, where each dictionary contains the following keys:
        - 'production_rate': the production rate of the well (in m3/day)
        - 'gas_lift': the gas lift of the well (in m3/day)
    - gas_capacity: the total gas capacity available for allocation (in m3/day)
    - alpha: the weight of the pheromone trail in the decision making process
    - beta: the weight of the heuristic value in the decision making process
    - evaporation_rate: the rate at which the pheromone trails evaporate
    - iteration_count: the number of iterations to run the algorithm for
    
    Returns:
    - A list of dictionaries representing the optimized allocation of gas to the wells, where each dictionary contains the following keys:
        - 'well_id': the ID of the well
        - 'allocation': the optimized allocation of gas to the well (in m3/day)
    """
    # Initialize the pheromone trails and heuristic values for each well
    pheromone_trails = np.ones(len(wells))
    heuristic_values = np.array([well['gas_lift'] / well['production_rate'] for well in wells])
    
    # Run the algorithm for the specified number of iterations
    for i in range(iteration_count):
        # Initialize the allocation for each well
        allocations = np.zeros(len(wells))
        
        # Allocate gas to each well using the ant colony algorithm
        remaining_gas = gas_capacity
        for j, well in enumerate(wells):
            # Calculate the allocation probability for the well
            allocation_probability = (pheromone_trails[j] ** alpha) * (heuristic_values[j] ** beta)
            allocation_probability /= np.sum((pheromone_trails ** alpha) * (heuristic_values ** beta))
            
            # Allocate gas to the well based on the allocation probability
            allocation = remaining_gas * allocation_probability
            allocations[j] = allocation
            remaining_gas -= allocation
              # Evaporate the pheromone trails based on the evaporation rate
        pheromone_trails *= (1 - evaporation_rate)
        
        # Update the pheromone trails based on the allocations
        for j, well in enumerate(wells):
            pheromone_trails[j] += allocations[j] / well['production_rate']
    
    # Return the optimized allocation of gas to the wells
    return [{'well_id': j, 'allocation': allocations[j]} for j in range(len(wells))]

In [34]:
# Define the test case
gas_capacity = gas_available
alpha = 1
beta = 2
evaporation_rate = 0.1
iteration_count = 50

# Optimize the allocation of gas to the wells
allocations = optimize_gas_allocation(wells, gas_capacity, alpha, beta, evaporation_rate, iteration_count)

# Print the optimized allocation
print(allocations)

[{'well_id': 0, 'allocation': 1.3330493605740525}, {'well_id': 1, 'allocation': 0.07130488163206354}, {'well_id': 2, 'allocation': 0.08862463953951208}, {'well_id': 3, 'allocation': 0.1543241559242013}, {'well_id': 4, 'allocation': 0.42734362956863764}, {'well_id': 5, 'allocation': 0.026293584482126018}, {'well_id': 6, 'allocation': 0.03351777556446701}, {'well_id': 7, 'allocation': 0.19768711924685928}, {'well_id': 8, 'allocation': 0.029203208757630366}, {'well_id': 9, 'allocation': 0.027557944828556984}]


In [35]:
sum = 0
for i in range(len(allocations)):
    sum+=allocations[i]['allocation']

sum

2.388906300118107